# Table of Contents
 <p>

In [2]:
import numpy as np

In [157]:
t_a = np.array([['a','m','1'],['a','m','2'],['b','n','1'],['b','n','1'],['b','n','1'],['b','n','2'],['a1','o','1']])

In [160]:
t_a

array([['a', 'm', '1'],
       ['a', 'm', '2'],
       ['b', 'n', '1'],
       ['b', 'n', '1'],
       ['b', 'n', '1'],
       ['b', 'n', '2'],
       ['a1', 'o', '1']], 
      dtype='<U2')

In [153]:
np.count_nonzerocols[np.where(cluster=='1')]

array(['a', 'b', 'b', 'b', 'a1'], 
      dtype='<U2')

In [161]:
#def VDM(a,b,index):
cols = t_a[:,0]
cluster = t_a[:,2]
    

In [131]:
from itertools import combinations

In [147]:
np.unique(cluster)

array(['1', '2'], 
      dtype='<U2')

In [158]:
m_u

(array(['a', 'a1', 'b'], 
       dtype='<U2'), array([1, 1, 5]))

In [203]:

def vdm(cols,clu):
    m_u = np.unique(cols,return_counts=True)
    vdm_value = np.zeros(len(list(combinations(m_u[0],2))))
    for k in np.unique(clu):
        m_ui = np.unique(cols[np.where(cluster==k)],return_counts=True)
        m_ui0,m_ui1 = m_ui
        array_diff = np.setdiff1d(m_u[0],m_ui[0])
        print(array_diff)
        m_ui1 = np.append(m_ui1,[0]*len(array_diff))
        m_ui0 =  np.append(m_ui0,array_diff)
        print(m_ui0,m_ui1,)
        s_mui = sorted(zip(m_ui0,m_ui1),key=lambda x:x[0])
        s_mui1 = np.array([y for x,y in s_mui])
        clu_rate = zip(m_u[0],s_mui1/m_u[1])    
        clu_sq = [np.square(c[0][1]-c[1][1]) for c in combinations(clu_rate,2)]
        print(clu_sq)
        vdm_value += np.array(clu_sq)
        print(vdm_value)
            #print((c[0][0],c[1][0]),np.square(c[0][1]-c[1][1]))
    ret = dict(zip(combinations(m_u[0],2),vdm_value))
    return ret


In [204]:
vdm(t_a[:,0],t_a[2])

[]
['a' 'a1' 'b'] [ 1.  1.  3.]
[0.25, 0.0625, 0.0625]
[ 0.25    0.0625  0.0625]
['a' 'a1' 'b']
['a' 'a1' 'b'] [0 0 0]
[0.0, 0.0, 0.0]
[ 0.25    0.0625  0.0625]
['a' 'a1' 'b']
['a' 'a1' 'b'] [0 0 0]
[0.0, 0.0, 0.0]
[ 0.25    0.0625  0.0625]


{('a', 'a1'): 0.25, ('a', 'b'): 0.0625, ('a1', 'b'): 0.0625}

In [ ]:
dict()

In [194]:
vdm

array([ 0.5  ,  0.125,  0.125])

In [191]:
vdm = np.zeros(len(list(combinations(m_u[0],2))))
vdm

array([ 0.,  0.,  0.])

In [183]:
len(list(combinations(m_u[0],2)))

3

In [130]:
list(clu_rate)

[('a', 0.5), ('a1', 0.0), ('b', 0.25)]

In [117]:
np.sort(m_ui1,order=m_u[0])

ValueError: Cannot specify order when the array has no fields.

In [115]:
s_mui

[('a', 1, 'a', 2), ('a1', 0, 'b', 4), ('b', 1, 'a1', 1)]

In [107]:
m_u[0]

array(['a', 'a1', 'b'], 
      dtype='<U2')

In [109]:
list(zip(s_mui[0],m_u[1],m_u[0]))

[(1, 2, 'a'), ('a', 1, 'a1')]

In [ ]:
np.sort(m_ui1,order=)

In [73]:
m_ui1

array([1, 1, 0])

array(['a', 'b'], 
      dtype='<U1')

In [30]:
for k in np.unique(cluster):
    

array(['a', 'b', 'c'], 
      dtype='<U1')